In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from tqdm.auto import tqdm
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
result_folder_name = 'results_crossmodal'
path = os.path.join(os.getcwd(), result_folder_name)
x = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
result_files = {f'./{result_folder_name}/{f}': f.split('_')[-1].replace('.csv','') for f in x}
for i, f in enumerate(x):
    print(f"{i}: {f}")

0: result_imgtotext_chinese-clip-vit-large-patch14.pkl.csv
1: result_imgtotext_clip-vit-large-patch14.csv
2: result_imgtotext_ColQwen2.5-3b-multilingual-v1.0.csv
3: result_imgtotext_ColQwen2.5-7b-multilingual-v1.0.csv
4: result_imgtotext_colqwen2.5-v0.2.csv
5: result_imgtotext_gme-Qwen2-VL-2B-Instruct.csv
6: result_imgtotext_gme-Qwen2-VL-7B-Instruct.csv
7: result_imgtotext_jina-embeddings-v4.csv
8: result_imgtotext_XLM-Roberta-Large-Vit-B-16Plus.csv
9: result_imgtotext_XLM-Roberta-Large-Vit-L-14.csv


In [3]:
file_name = 'result_colqwen2_cleaxned.csv'
native_source = pd.read_csv('captions_index_key_native.csv')
lang_type = pd.read_csv('commoncrawl.csv')

In [4]:
df = pd.read_csv(f'./{result_folder_name}/{x[0]}')

# Functions

In [ ]:
def calculate_lbkl(df: pd.DataFrame, epsilon: float = 1e-9, rank: int = 5, debug: bool = False, variant='original') -> float:
    """
    Calculates the Language Bias Kullback–Leibler Divergence (LBKL)
    from a DataFrame, focusing on result_rank 1.

    Assumes P_native(x) = 0.5 and P_en(x) = 0.5 for all queries.
    Uses an epsilon to handle zero probabilities in Q to avoid infinite results.

    Args:
        df: A pandas DataFrame with columns 'query_id', 'result_rank',
            'result_id', 'native', 'score'.
            'native' column is boolean (True for native, False for English).
        epsilon: A small value added to zero proportions in Q to avoid log(0).
        rank: Maximum rank to consider
        debug: Whether to print debug information
        variant: 'original' for unweighted, 'rank' for logarithmic decay, 'linear' for linear decay

    Returns:
        The calculated LBKL score.
    """
    epsilon = 1e-9

    P_native = 0.5#
    P_en = 0.5

    query_groups = df.groupby('query_id')

    total_lbkl_sum = 0
    q = 0

    for query_id, group_full in query_groups:
        q += 1
        group = group_full.sort_values(by='result_rank')
        group = group_full.head(rank) # Limit to the specified rank

        # Count native and English results across all ranks for this query
        num_native_total = group['native'].sum()
        num_english_total = len(group) - num_native_total
        total_results = len(group)
        
        if variant == 'rank':
            weighted_native_sum = 0
            weighted_english_sum = 0
            total_weight_sum = 0

            for idx, row in group.iterrows():
                i = row['result_rank']  # rank position (1-based)
                w_i = 1 / np.log2(i + 1)  # logarithmic discount weight
                total_weight_sum += w_i
                
                # Add weighted contributions
                if row['native']:  # Thai/native document
                    weighted_native_sum += w_i
                else:  # English document
                    weighted_english_sum += w_i
            
            # Calculate weighted proportions Q'(x)
            if total_weight_sum > 0:
                Q_native_x = weighted_native_sum / total_weight_sum
                Q_en_x = weighted_english_sum / total_weight_sum
            else:
                Q_native_x = 0.0
                Q_en_x = 0.0
        else: # original lbkl 
            # Calculate proportions Q(x) based on all retrieved results
            if total_results > 0:
                Q_native_x = num_native_total / total_results
                Q_en_x = num_english_total / total_results
            else:
                Q_native_x = 0.0
                Q_en_x = 0.0

        # Calculate KL divergence terms
        kl_term_native = 0.0
        if P_native > 0:
            if Q_native_x == 0:
                kl_term_native = P_native * np.log(P_native / epsilon)  # Handle zero Q_native_x
            else:
                kl_term_native = P_native * np.log(P_native / Q_native_x)

        kl_term_en = 0.0
        if P_en > 0:
            if Q_en_x == 0:
                kl_term_en = P_en * np.log(P_en / epsilon) # Handle zero Q_en_x
            else:
                kl_term_en = P_en * np.log(P_en / Q_en_x)

        lbkl = (kl_term_native + kl_term_en)
        total_lbkl_sum += lbkl

        if debug:
            print(f"Query ID: {query_id}, "
                  f"Native Count: {num_native_total}, "
                  f"English Count: {num_english_total}, "
                  f"Total Results: {len(group)}, "
                  f"Q_native_x: {Q_native_x:.4f}, "
                  f"Q_en_x: {Q_en_x:.4f}, "
                  f"KL Native: {kl_term_native:.4f}, "
                  f"KL English: {kl_term_en:.4f}, "
                  f"LBKL: {lbkl:.4f}")

    lbkl_score = total_lbkl_sum / q
    return lbkl_score

In [5]:
# Calculate accuracy for each model at rank 1
def calculate_rank1_accuracy(df, k=5):
    correct_at_k = 0
    total_queries = 0
    
    for query_id in df['query_id'].unique():
        query_results = df[df['query_id'] == query_id].head(k)
        # Check if any result in top k is the correct one (result_id == query_id)
        if any(query_results['result_id'] == query_id):
            correct_at_k += 1
        total_queries += 1
    accuracy = correct_at_k / total_queries if total_queries > 0 else 0
    return accuracy, correct_at_k, total_queries

In [6]:
def calculate_dcg(relevances, k=None):
    """Calculate DCG@k"""
    if k is not None:
        relevances = relevances[:k]
    
    dcg = 0
    for i, rel in enumerate(relevances):
        dcg += rel / np.log2(i + 2)  # i+2 because positions start at 1
    return dcg

def calculate_ndcg(actual_relevances, k=None):
    """Calculate NDCG@k"""
    dcg = calculate_dcg(actual_relevances, k)
    ideal_relevances = sorted(actual_relevances, reverse=True)
    idcg = calculate_dcg(ideal_relevances, k)
    
    if idcg == 0:
        return 0
    return dcg / idcg

def calculate_ndcg_for_results(df, k=10):
    """Calculate NDCG@k for your retrieval results"""
    ndcg_scores = []
    
    for query_id in df['query_id'].unique():
        query_results = df[df['query_id'] == query_id].sort_values('result_rank')
        
        # Create relevance scores (1 for correct match, 0 for incorrect)
        relevances = (query_results['query_id'] == query_results['result_id']).astype(int).values
        
        ndcg = calculate_ndcg(relevances, k)
        ndcg_scores.append(ndcg)
    
    return np.mean(ndcg_scores)

# Each metric

## LBKL/DLBKL

In [11]:
ranks_to_analyze = [1, 5, 10, 15, 25, 50, 99]
lbkl_results = []
rlbkl_results = []

# Process each model
for file_path, model_name in result_files.items():
    # Load the model results
    df_model = pd.read_csv(file_path)
    df_model = df_model.merge(native_source.drop(columns=['key']), how='left', left_on='query_id', right_on='index')
    df_model = df_model.merge(lang_type, how='left', left_on='lang', right_on='Id')
    df_model['native'] = df_model['lang_type'].apply(lambda x: x == 'low')
    model_scores = {'Model': model_name}
    
    for rank in ranks_to_analyze:
        lbkl_score = calculate_lbkl(df_model, rank=rank)
        model_scores[f'LBKL@{rank}'] = round(lbkl_score, 4)
        rlbkl_score = calculate_lbkl(df_model, rank=rank, variant='rank')
        model_scores[f'dLBKL@{rank}'] = round(rlbkl_score, 4)
    
    lbkl_results.append(model_scores)
    rlbkl_results.append(model_scores)
    
    print(f"Processed {model_name}")
        
# Create DataFrame
lbkl_df = pd.DataFrame(lbkl_results)
rlbkl_df = pd.DataFrame(rlbkl_results)

# Display results
print()
print("dLBKL Scores by Model and Rank:")
print("=" * 80)
print(lbkl_df.to_string(index=False, float_format='%.4f'))

Processed chinese-clip-vit-large-patch14.pkl
Processed clip-vit-large-patch14
Processed ColQwen2.5-3b-multilingual-v1.0
Processed ColQwen2.5-7b-multilingual-v1.0
Processed colqwen2.5-v0.2
Processed gme-Qwen2-VL-2B-Instruct
Processed gme-Qwen2-VL-7B-Instruct
Processed jina-embeddings-v4
Processed XLM-Roberta-Large-Vit-B-16Plus
Processed XLM-Roberta-Large-Vit-L-14

dLBKL Scores by Model and Rank:
                             Model  LBKL@1  dLBKL@1  LBKL@5  dLBKL@5  LBKL@10  dLBKL@10  LBKL@15  dLBKL@15  LBKL@25  dLBKL@25  LBKL@50  dLBKL@50  LBKL@99  dLBKL@99
chinese-clip-vit-large-patch14.pkl  9.6685   9.6685  6.2203   6.2288   5.0457    5.0548   4.4233    4.4321   3.7929    3.7982   3.1720    3.1684   2.5816    2.5697
            clip-vit-large-patch14  9.6685   9.6685  6.9036   6.9107   5.6732    5.6841   4.9647    4.9761   4.1713    4.1816   3.2449    3.2493   2.6579    2.6519
   ColQwen2.5-3b-multilingual-v1.0  9.6685   9.6685  2.1381   2.1640   0.7919    0.8172   0.4135    0.4357   0

## Accuracy

In [39]:
# Compare accuracy across all models
K = 1
print(f"Accuracy@{K} Comparison:")
print("=" * 60)
print(f"{'Model':<30} {'Correct':<8} {'Total':<8} {'Accuracy':<10}")
print("-" * 60)
model_results = {}

for file_path, model_name in result_files.items():
    try:
        df = pd.read_csv(file_path)
        accuracy, correct, total = calculate_rank1_accuracy(df, k=K)
        model_results[model_name] = {
            'accuracy': accuracy,
            'correct': correct,
            'total': total
        }
        print(f"{model_name:<30} {correct:<8} {total:<8} {accuracy:<10.4f}")
    except FileNotFoundError:
        print(f"{model_name:<30} {'N/A':<8} {'N/A':<8} {'N/A':<10}")

print("-" * 60)

Accuracy@1 Comparison:
Model                          Correct  Total    Accuracy  
------------------------------------------------------------
lora8                          2103     3600     0.5842    
------------------------------------------------------------


## NDCG

In [19]:
# Usage with your existing code:
NDCG_AT = 99

# Collect NDCG results
model_ndcg_results = {}
for file_path, model_name in result_files.items():
    try:
        df = pd.read_csv(file_path)
        ndcg_10 = calculate_ndcg_for_results(df, k=NDCG_AT)
        model_ndcg_results[model_name] = ndcg_10
    except FileNotFoundError:
        print(f"{model_name}: File not found")

        
# Print the comparison table
print(f"\nNDCG@{NDCG_AT} Comparison:")
print("=" * 60)
print(f"{'Model':<40} {f'NDCG@{NDCG_AT}':<10}")
print("-" * 60)
for model_name, ndcg_score in model_ndcg_results.items():
    print(f"{model_name:<40} {ndcg_score:<10.4f}")
print("-" * 60)        


NDCG@99 Comparison:
Model                                    NDCG@99   
------------------------------------------------------------
clip-vit-large-patch14                   0.3067    
ColQwen2.5-3b-multilingual-v1.0          0.7566    
ColQwen2.5-7b-multilingual-v1.0          0.7978    
colqwen2.5-v0.2                          0.6246    
gme-Qwen2-VL-2B-Instruct                 0.6020    
gme-Qwen2-VL-7B-Instruct                 0.6822    
gme7B                                    0.7142    
siglip-so400m-patch14-384                0.5115    
------------------------------------------------------------


# Summarize Single Model

In [4]:
for i, f in enumerate(x):
    print(f"{i}: {f}")

0: result_imgtotext_chinese-clip-vit-large-patch14.pkl.csv
1: result_imgtotext_clip-vit-large-patch14.csv
2: result_imgtotext_ColQwen2.5-3b-multilingual-v1.0.csv
3: result_imgtotext_ColQwen2.5-7b-multilingual-v1.0.csv
4: result_imgtotext_colqwen2.5-v0.2.csv
5: result_imgtotext_gme-Qwen2-VL-2B-Instruct.csv
6: result_imgtotext_gme-Qwen2-VL-7B-Instruct.csv
7: result_imgtotext_jina-embeddings-v4.csv
8: result_imgtotext_XLM-Roberta-Large-Vit-B-16Plus.csv
9: result_imgtotext_XLM-Roberta-Large-Vit-L-14.csv


In [6]:
model_idx = 0
model_name = x[model_idx].replace('result_', '').replace('_cleaned.csv', '')
df = pd.read_csv(f'./{result_folder_name}/{x[model_idx]}')
# df = df.merge(native_source.drop(columns=['key']), how='left', left_on='query_id', right_on='index')
df = df.merge(lang_type, how='left', left_on='lang', right_on='Id')
df['native'] = df['lang_type'].apply(lambda x: x == 'low')
print(model_name)
df.head(20)

imgtotext_chinese-clip-vit-large-patch14.pkl.csv


,query_id,result_rank,result_id,lang,caption_idx,score,Language,Id,dist,lang_type,native
0,0,1,2024,ja,1,0.3290,Japanese,ja,4.9152,medium,False
1,0,2,1330,ja,2,0.3082,Japanese,ja,4.9152,medium,False
2,0,3,1923,ja,2,0.3058,Japanese,ja,4.9152,medium,False
3,0,4,3510,ja,1,0.3005,Japanese,ja,4.9152,medium,False
4,0,5,1281,ja,2,0.2982,Japanese,ja,4.9152,medium,False
5,0,6,2233,ja,1,0.2843,Japanese,ja,4.9152,medium,False
6,0,7,1841,ja,1,0.2826,Japanese,ja,4.9152,medium,False
7,0,8,842,zh,1,0.2820,Chinese-Simpl.,zh,4.8778,medium,False
8,0,9,2613,ja,1,0.2812,Japanese,ja,4.9152,medium,False
9,0,10,2024,ja,2,0.2809,Japanese,ja,4.9152,medium,False


In [37]:
df[(df['result_rank'] <= 5) & (df['query_id'] == df['result_id'])]

,query_id,result_rank,result_id,lang,score,native,Language,Id,dist,lang_type
0,0,1,0,en,225.3604,False,English,en,43.9499,high
1,0,2,0,el,211.4246,True,Greek,el,0.5651,low
100,1,1,1,vi,204.8562,False,Vietnamese,vi,1.0213,medium
101,1,2,1,it,204.5964,False,Italian,it,2.4060,medium
102,1,3,1,vi,201.3790,False,Vietnamese,vi,1.0213,medium
...,...,...,...,...,...,...,...,...,...,...
118903,1189,4,1189,te,216.0825,True,Telugu,te,0.0213,low
202500,2025,1,2025,da,204.4172,True,Danish,da,0.4792,low
220404,2204,5,2204,da,177.9981,True,Danish,da,0.4792,low
246200,2462,1,2462,da,165.4088,True,Danish,da,0.4792,low


In [10]:
AT_RANK = 99
ACC_AT = 5

# Calculate metrics
accuracy = round(calculate_rank1_accuracy(df, k=ACC_AT)[0], 4)
ndcg = round(calculate_ndcg_for_results(df, k=AT_RANK), 4)
lbkl = round(calculate_lbkl(df, rank=AT_RANK), 4)
dlbkl = round(calculate_lbkl(df, rank=AT_RANK, variant='rank'), 4)

print(f"\n{'='*70}")
print(f"{'QUANTITATIVE SUMMARY':^70}")
print(f"{'='*70}")
print(f"Model: {model_name}")
print(f"{'-'*70}")
print(f"{'Metric':<20} {'Value':<15} {'Rank':<10}")
print(f"{'-'*70}")
print(f"{'Accuracy@' + str(ACC_AT):<20} {accuracy:<15.4f} {'@' + str(ACC_AT):<10}")
print(f"{'NDCG@' + str(AT_RANK):<20} {ndcg:<15.4f} {'@' + str(AT_RANK):<10}")
print(f"{'LBKL@' + str(AT_RANK):<20} {lbkl:<15.4f} {'@' + str(AT_RANK):<10}")
print(f"{'DLBKL@' + str(AT_RANK):<20} {dlbkl:<15.4f} {'@' + str(AT_RANK):<10}")
print(f"{'='*70}")


                         QUANTITATIVE SUMMARY                         
Model: imgtotext_chinese-clip-vit-large-patch14.pkl.csv
----------------------------------------------------------------------
Metric               Value           Rank      
----------------------------------------------------------------------
Accuracy@5           0.3550          @5        
NDCG@99              0.2953          @99       
LBKL@99              2.5816          @99       
DLBKL@99             2.5697          @99       


In [7]:
AT_RANK = 99
lbkl = round(calculate_lbkl(df, rank=AT_RANK), 4)
dlbkl = round(calculate_lbkl(df, rank=AT_RANK, variant='rank'), 4)
print(f"{'LBKL@' + str(AT_RANK):<20} {lbkl:<15.4f} {'@' + str(AT_RANK):<10}")
print(f"{'DLBKL@' + str(AT_RANK):<20} {dlbkl:<15.4f} {'@' + str(AT_RANK):<10}")

LBKL@99              0.0833          @99       
DLBKL@99             0.1116          @99       


# Summarize All Model

In [7]:
AT_RANK = 10
ACC_AT = 5

# Initialize results storage
all_model_results = []

print("Processing all models...")
print("=" * 80)

# Process each model
for file_path, model_name in result_files.items():
    try:
        # Load and preprocess data
        df = pd.read_csv(file_path)
        # df = df.merge(native_source.drop(columns=['key']), how='left', left_on='query_id', right_on='index')
        df = df.merge(lang_type, how='left', left_on='lang', right_on='Id')
        df['native'] = df['lang_type'].apply(lambda x: x == 'low')
        
        # Calculate all metrics
        accuracy = round(calculate_rank1_accuracy(df, k=ACC_AT)[0], 4)
        ndcg = round(calculate_ndcg_for_results(df, k=AT_RANK), 4)
        lbkl = round(calculate_lbkl(df, rank=AT_RANK), 4)
        dlbkl = round(calculate_lbkl(df, rank=AT_RANK, variant='rank'), 4)
        
        # Store results
        model_results = {
            'Model': model_name,
            f'Accuracy@{ACC_AT}': accuracy,
            f'NDCG@{AT_RANK}': ndcg,
            f'LBKL@{AT_RANK}': lbkl,
            f'dLBKL@{AT_RANK}': dlbkl
        }
        all_model_results.append(model_results)
        
        print(f"✓ Processed {model_name}")
        
    except FileNotFoundError:
        print(f"✗ {model_name}: File not found")
    except Exception as e:
        print(f"✗ {model_name}: Error - {str(e)}")
    # break

# Create summary DataFrame
summary_df = pd.DataFrame(all_model_results)

# Display beautiful summary
print("\n" + "=" * 80)
print(f"{'ALL MODELS PERFORMANCE SUMMARY':^80}")
print("=" * 80)

if not summary_df.empty:
    # Sort by accuracy (descending)
    summary_df_sorted = summary_df.sort_values(f'Accuracy@{ACC_AT}', ascending=False)
    
    print(f"{'Model':<40} {'Acc@' + str(ACC_AT):<10} {'NDCG@' + str(AT_RANK):<12} {'LBKL@' + str(AT_RANK):<12} {'dLBKL@' + str(AT_RANK):<12}")
    print("-" * 80)
    
    for _, row in summary_df_sorted.iterrows():
        print(f"{row['Model']:<40} {row[f'Accuracy@{ACC_AT}']:<10.4f} "
              f"{row[f'NDCG@{AT_RANK}']:<12.4f} {row[f'LBKL@{AT_RANK}']:<12.4f} "
              f"{row[f'dLBKL@{AT_RANK}']:<12.4f}")
    
    print("-" * 80)
    
    # Best performing models
    best_accuracy = summary_df_sorted.iloc[0]
    best_ndcg = summary_df.loc[summary_df[f'NDCG@{AT_RANK}'].idxmax()]
    best_lbkl = summary_df.loc[summary_df[f'LBKL@{AT_RANK}'].idxmin()]  # Lower is better for LBKL
    
    print("\n🏆 BEST PERFORMERS:")
    print(f"Best Accuracy@{ACC_AT}: {best_accuracy['Model']} ({best_accuracy[f'Accuracy@{ACC_AT}']:.4f})")
    print(f"Best NDCG@{AT_RANK}: {best_ndcg['Model']} ({best_ndcg[f'NDCG@{AT_RANK}']:.4f})")
    print(f"Best LBKL@{AT_RANK}: {best_lbkl['Model']} ({best_lbkl[f'LBKL@{AT_RANK}']:.4f})")
    
    # Summary statistics
    print(f"\n📊 SUMMARY STATISTICS:")
    print(f"Total models processed: {len(summary_df)}")
    print(f"Average Accuracy@{ACC_AT}: {summary_df[f'Accuracy@{ACC_AT}'].mean():.4f}")
    print(f"Average NDCG@{AT_RANK}: {summary_df[f'NDCG@{AT_RANK}'].mean():.4f}")
    print(f"Average LBKL@{AT_RANK}: {summary_df[f'LBKL@{AT_RANK}'].mean():.4f}")
    
else:
    print("No models were successfully processed.")

print("=" * 80)

Processing all models...
✓ Processed chinese-clip-vit-large-patch14.pkl
✓ Processed clip-vit-large-patch14
✓ Processed ColQwen2.5-3b-multilingual-v1.0
✓ Processed ColQwen2.5-7b-multilingual-v1.0
✓ Processed colqwen2.5-v0.2
✓ Processed gme-Qwen2-VL-2B-Instruct
✓ Processed gme-Qwen2-VL-7B-Instruct
✓ Processed jina-embeddings-v4
✓ Processed XLM-Roberta-Large-Vit-B-16Plus
✓ Processed XLM-Roberta-Large-Vit-L-14

                         ALL MODELS PERFORMANCE SUMMARY                         
Model                                    Acc@5      NDCG@10      LBKL@10      dLBKL@10    
--------------------------------------------------------------------------------
gme-Qwen2-VL-7B-Instruct                 0.9792     0.7699       1.3706       1.4203      
jina-embeddings-v4                       0.9717     0.7753       0.9152       0.9506      
XLM-Roberta-Large-Vit-B-16Plus           0.9675     0.7908       0.1098       0.1248      
gme-Qwen2-VL-2B-Instruct                 0.9672     0.7166     